## MAG processing
We used atlas on around 900 datasets, after checking, 338 of those have good enough SRA measurements:
Need to re-check, bc i did something wrong, I think 

- WGS
- metagenomic
- random amplification
- paired libraries
- more than 1GB sequencing
- pig gut metagenome == organism


For these, link the atlas output folders, and do following:
- Do taxonomy using sourmash, probably just do it for all of them, or copy tax file
- dereplicate using drep
- Create signatures for the ones that don't have a signature yet. 
- Get the quality stats for them somewhere local
- do a fastmultigather for these against high quality metagenomes. 



In [ ]:
# use the 339 'good' datasets for now. 
cp atlas/*/genomes/genomes/*.fasta ./genomes/

# concat all tax
mamba activate csvtk
csvtk concat atlas/atlas*/genomes/taxonomy/gtdb_taxonomy.tsv -t > 240705_genome_taxonomy.tsv

# concat quality info
mamba activate csvtk
csvtk concat atlas/atlas*/genomes/genome_quality.tsv -t > 240705_genome_quality.tsv

In [ ]:
# deduplicate using drep 
# use external info from checkm: https://drep.readthedocs.io/en/master/advanced_use.html#using-external-genome-quality-information
# make a completeness sheet for drep https://github.com/MrOlm/drep/issues/220
awk -F'\t' 'BEGIN {OFS=","} {print $1, $2, $3}' quality_report.tsv > new_file_name.csv
# In the new file convert headings to: genome,completeness,contamination

# use drep (retains 831 genomes)
srun --account=ctbrowngrp -p bmh -J sketch -t 5:00:00 -c 32 --mem=50gb --pty bash

mamba activate drep
dRep dereplicate \
drep.999 \
--genomeInfo drepqual.csv \
-p 32 \
-g ./genomes/*.fasta \
-pa 0.9 -sa 0.999 -nc 0.30 -cm larger \
-comp 50 -con 10 -l 1000 


In [ ]:
# Create MAG signatures for fastgather usage:
# Do only for all genomes, later split into 95%, 99% and 99.9% ANI
module load parallel
mamba activate branchwater

for f in *.fasta
do
echo sourmash sketch dna \
-p k=21,scaled=1000,k=31,scaled=1000 \
$f --name ${f%.fasta*} -o ../../sourmash/sketches/MAGs/${f%.fasta*}.zip
done | parallel -j 32

In [ ]:
# Make file lists for dereplicated MAGs and replace .fasta with .zip
sed -i 's/\.fasta/\.zip/g' drep95.txt

# Concatenate all signatures --from-file
sourmash sig cat --from-file ../../../MAGs/genome_lists/drep95.txt -o ../MAGs.drep95.zip
sourmash sig cat --from-file ../../../MAGs/genome_lists/drep99.txt -o ../MAGs.drep99.zip
sourmash sig cat --from-file ../../../MAGs/genome_lists/drep999.txt -o ../MAGs.drep999.zip

sourmash sig collect --from-file ../../../MAGs/genome_lists/drep999.txt -o ../MAGs.drep999.sqlmf
sourmash sig collect --from-file ../../../MAGs/genome_lists/drep99.txt -o ../MAGs.drep99.sqlmf
sourmash sig collect --from-file ../../../MAGs/genome_lists/drep95.txt -o ../MAGs.drep95.sqlmf

In [ ]:
# after fastgather, make picklist and create new db, do fg against this
sourmash sig cat \
../../sketches/MAGs.drep999.zip \
/group/ctbrowngrp/sourmash-db/gtdb-rs214/gtdb-rs214-k21.zip \
-k 21 --picklist SRR5976183.pig.csv:match_md5:md5 -o SRR5976183.concat.zip

sourmash scripts fastgather \
{input.sig} {input.db} \
        -k {KSIZE} --scaled 1000 \
        -c {threads} -o {output.csv}

In [ ]:
# some sort of gather
rule sourmash_gather_all:
    input:  
        sig = "/home/ctbrown/scratch2/2023-swine-usda/outputs.swine-x-reps/{sample}.subtract.sig.gz",
        picklist = "sourmash/fastgather/concat/{sample}.csv"
    output: 
        csv = "sourmash/gather_out/MAGs_gtdbk_vir/{sample}.csv"
    resources:
        mem_mb=25000
    conda: 
        "sourmash"
    shell:
        """
        sourmash gather {input.sig} atlas/MAGs/all-MAGs_21.zip \
        /group/ctbrowngrp/sourmash-db/gtdb-rs214/gtdb-rs214-k21.zip \
        /virsorter2/all-votu_21.zip 
        -k 21 --scaled 10000 -o {output.csv} \
        --picklist {input.picklist}:match_md5:md5
        """


In [ ]:
## Run iphop to link viruses to hosts, from the same samples
